In [1]:
import pandas as pd
import numpy as np
import swifter

from common.preprocessing.dataframe import *
import json
from common.bio.constants import ID_TO_AMINO_ACID

Config

In [2]:
LEVEL = "Level_4"
SEQUENCE_LENGTH = 512
MIN_SEQUENCE_LENGTH = 80
ROOT = "../../data/protein/classification/"
file_path = ROOT + "data_sources/uniprot-reviewed%3Ayes.tab.gz"
save_path = ROOT + "sample_" + str(SEQUENCE_LENGTH)
os.makedirs(save_path, exist_ok = True)

# Loading data

In [3]:
data = pd.read_csv(file_path, sep='\t', header=(0), skipinitialspace=True).drop("Entry", axis=1)
data = data.rename(columns={"Sequence" :"sequence"})
data.sequence = data.sequence.str.strip()

In [4]:
data.head()

,EC number,sequence
0,3.1.3.48,MARAGGNCGVWRSLVLLGLYGCSVVRAAGTSVTVDRHAPASSYEFS...
1,3.1.3.48,MKPAARETRTPPRSPGLRWALLPLLLLLRQGQVLCAGAAPNPIFDI...
2,3.1.3.48,MRTLGTCLVTLAGLLLTAAGETFSGGCLFDEPYSTCGYSQADEDDF...
3,NaN,MRRPRRPGGPAGCGGSEGSGGLRLLVCLLLLSGRPGGCSAISAHGC...
4,3.1.3.48,MGHLPTGIHGARRLLPLLWLFVLFKNATAFHVTVQDDNNIVVSLEA...


# Data quality checks

In [5]:
data[data["sequence"].str.contains(" ")]

,EC number,sequence


In [6]:
len(data)

557713

Limit to length sequences (for efficiency) & Also remove sequences without labels

In [7]:
processed_data = data[(data["sequence"].str.len() <= SEQUENCE_LENGTH) & (data["sequence"].str.len() >= MIN_SEQUENCE_LENGTH) &
                      (data["EC number"].notnull()) & (data["sequence"].str.startswith("M"))]

In [8]:
def remove_classes(row, separator, to_remove):
    split_row = row.split(separator)
    new_column = separator.join([ec for ec in split_row if not to_remove in ec])
    return new_column
processed_data["EC number"] = processed_data["EC number"].apply(remove_classes, args=(";", "-"))
processed_data = processed_data[processed_data["EC number"] != ""]

c:\users\donatas\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
len(processed_data)

178507

Splitting enzymes into multiple rows that belong to multiple classes 

In [10]:
#processed_data = split_dataframe_list_to_rows(processed_data, "EC number", ";")
processed_data["EC number"] = processed_data["EC number"].str.strip()

Filter out B,O,U,X,Z amino acids

In [11]:
for amino_acid in ['B','O','U','X','Z' ]:
    processed_data = processed_data[~processed_data["sequence"].str.contains(amino_acid)]

In [12]:
len(processed_data)

178120

In [13]:
processed_data.head()

,EC number,sequence
12,6.3.4.4,MSGTRASNDRPPGAGGVKRGRLQQEAAATGSRVTVVLGAQWGDEGK...
13,6.3.4.4,MSGTRASNDRPPGTGGVKRGRLQQEAAATGSRVTVVLGAQWGDEGK...
14,6.3.4.4,MAFAETYPAASSLPNGDCGRPRARPGGNRVTVVLGAQWGDEGKGKV...
15,6.3.4.4,MSISESSPAATSLPNGDCGRPRARSGGNRVTVVLGAQWGDEGKGKV...
18,1.11.2.3,MGSKTEMMERDAMATVAPYAPVTYHRRARVDLDDRLPKPYMPRALQ...


In [14]:
def countAminoAcid(data, amino_acid):
    return len([ 1 for index, val in data.sequence.iteritems() if amino_acid in val ])


In [15]:
countAminoAcid(processed_data, 'Z')

0

## Generating labels and features

Parsing labels

In [16]:
def generateLabel(item, index):
    return ".".join(item.split(".")[0:index])  

In [17]:
processed_data_level = processed_data[["sequence", "EC number"]]
levels = []
for i in range(1,5):
    levels.append("Level_"+str(i))
    processed_data["Level_"+str(i)] = [ generateLabel(item,i) for item in processed_data["EC number"] ]
processed_data = processed_data.drop("EC number", axis=1)
processed_data.head()

,sequence,Level_1,Level_2,Level_3,Level_4
12,MSGTRASNDRPPGAGGVKRGRLQQEAAATGSRVTVVLGAQWGDEGK...,6,6.3,6.3.4,6.3.4.4
13,MSGTRASNDRPPGTGGVKRGRLQQEAAATGSRVTVVLGAQWGDEGK...,6,6.3,6.3.4,6.3.4.4
14,MAFAETYPAASSLPNGDCGRPRARPGGNRVTVVLGAQWGDEGKGKV...,6,6.3,6.3.4,6.3.4.4
15,MSISESSPAATSLPNGDCGRPRARSGGNRVTVVLGAQWGDEGKGKV...,6,6.3,6.3.4,6.3.4.4
18,MGSKTEMMERDAMATVAPYAPVTYHRRARVDLDDRLPKPYMPRALQ...,1,1.11,1.11.2,1.11.2.3


## Level label processing

Converting unique label to int

In [18]:
def class_to_number(df, column):
    classes = df[column].sort_values().unique()
    print ("There are {} unique classes for level {}".format(len(classes), LEVEL))
    indexToClass = {i:classes[i] for i in range (len(classes))}
    classToIndex = {classes[i]:i for i in range (len(classes))}
    return indexToClass, classToIndex

Filterring out labels with little number of examples

In [19]:
import copy
levels_copy = copy.deepcopy(levels)
levels_copy.remove(LEVEL)

In [20]:
MIN_NUM_EXAMPLES = 25
processed_data_level = processed_data.groupby([LEVEL]).filter(lambda x: x[LEVEL].count() > MIN_NUM_EXAMPLES)

In [21]:
#processed_data_level = processed_data_level.drop(levels_copy, axis=1)
indexToClass, classToIndex = class_to_number(processed_data_level, LEVEL)

There are 769 unique classes for level Level_4


In [22]:
full_path = os.path.join(save_path, "classToIndex.json")
with open(full_path, 'w') as outfile:
    json.dump(classToIndex, outfile) 

In [23]:
processed_data_level["Label"] = [ classToIndex[val] for index, val in processed_data_level[LEVEL].iteritems()]

c:\users\donatas\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
path = os.path.join(save_path, "data.csv")
processed_data_level.to_csv(path, sep='\t', index=None)

In [25]:
processed_data_level = processed_data_level.drop(LEVEL, axis=1)

In [26]:
processed_data_level.head()

,sequence,Level_1,Level_2,Level_3,Label
12,MSGTRASNDRPPGAGGVKRGRLQQEAAATGSRVTVVLGAQWGDEGK...,6,6.3,6.3.4,758
13,MSGTRASNDRPPGTGGVKRGRLQQEAAATGSRVTVVLGAQWGDEGK...,6,6.3,6.3.4,758
14,MAFAETYPAASSLPNGDCGRPRARPGGNRVTVVLGAQWGDEGKGKV...,6,6.3,6.3.4,758
15,MSISESSPAATSLPNGDCGRPRARSGGNRVTVVLGAQWGDEGKGKV...,6,6.3,6.3.4,758
34,MQFKNILVVCIGNICRSPMAEYLLKQNYPQLTIHSAGISGMIGYSA...,3,3.1,3.1.3,441


Train/Test/Val

#### Train/test split

In [27]:
def train_val_test_split(data, val=0.2, test=0.1):
    train_split = int((1-val-test)*len(data))
    val_split = int((1-test)*len(data))
    train, val, test = np.split(data.sample(frac=1), [train_split, val_split])
    print("Full: {} | Train: {} | Validation: {} | Test: {}".format(len(data), len(train), len(val), len(test)))
    print("Train: {:.2f}% | Validation: {:.2f}% | Test: {:.2f}%".format(len(train)/len(data)*100, 
                                                                        len(val)/len(data)*100, 
                                                                        len(test)/len(data)*100))
    return train, val, test

In [28]:
data = processed_data_level.groupby("Label")
train, val, test = None, None, None
for group_id in range(len(data.groups)):
    train_g, val_g, test_g  = train_val_test_split(data.get_group(group_id))
    if train is None:
        train, val, test = train_g, val_g, test_g 
    else:
        train = train.append(train_g)
        val = val.append(val_g)
        test = test.append(test_g)
len(train), len(val), len(test)

Full: 187 | Train: 130 | Validation: 38 | Test: 19
Train: 69.52% | Validation: 20.32% | Test: 10.16%
Full: 45 | Train: 31 | Validation: 9 | Test: 5
Train: 68.89% | Validation: 20.00% | Test: 11.11%
Full: 187 | Train: 130 | Validation: 38 | Test: 19
Train: 69.52% | Validation: 20.32% | Test: 10.16%
Full: 41 | Train: 28 | Validation: 8 | Test: 5
Train: 68.29% | Validation: 19.51% | Test: 12.20%
Full: 31 | Train: 21 | Validation: 6 | Test: 4
Train: 67.74% | Validation: 19.35% | Test: 12.90%
Full: 159 | Train: 111 | Validation: 32 | Test: 16
Train: 69.81% | Validation: 20.13% | Test: 10.06%
Full: 58 | Train: 40 | Validation: 12 | Test: 6
Train: 68.97% | Validation: 20.69% | Test: 10.34%
Full: 38 | Train: 26 | Validation: 8 | Test: 4
Train: 68.42% | Validation: 21.05% | Test: 10.53%
Full: 51 | Train: 35 | Validation: 10 | Test: 6
Train: 68.63% | Validation: 19.61% | Test: 11.76%
Full: 44 | Train: 30 | Validation: 9 | Test: 5
Train: 68.18% | Validation: 20.45% | Test: 11.36%
Full: 220 | Trai

Train: 69.33% | Validation: 20.00% | Test: 10.67%
Full: 423 | Train: 296 | Validation: 84 | Test: 43
Train: 69.98% | Validation: 19.86% | Test: 10.17%
Full: 26 | Train: 18 | Validation: 5 | Test: 3
Train: 69.23% | Validation: 19.23% | Test: 11.54%
Full: 54 | Train: 37 | Validation: 11 | Test: 6
Train: 68.52% | Validation: 20.37% | Test: 11.11%
Full: 94 | Train: 65 | Validation: 19 | Test: 10
Train: 69.15% | Validation: 20.21% | Test: 10.64%
Full: 65 | Train: 45 | Validation: 13 | Test: 7
Train: 69.23% | Validation: 20.00% | Test: 10.77%
Full: 256 | Train: 179 | Validation: 51 | Test: 26
Train: 69.92% | Validation: 19.92% | Test: 10.16%
Full: 332 | Train: 232 | Validation: 66 | Test: 34
Train: 69.88% | Validation: 19.88% | Test: 10.24%
Full: 80 | Train: 56 | Validation: 16 | Test: 8
Train: 70.00% | Validation: 20.00% | Test: 10.00%
Full: 51 | Train: 35 | Validation: 10 | Test: 6
Train: 68.63% | Validation: 19.61% | Test: 11.76%
Full: 780 | Train: 546 | Validation: 156 | Test: 78
Train: 

Full: 96 | Train: 67 | Validation: 19 | Test: 10
Train: 69.79% | Validation: 19.79% | Test: 10.42%
Full: 101 | Train: 70 | Validation: 20 | Test: 11
Train: 69.31% | Validation: 19.80% | Test: 10.89%
Full: 161 | Train: 112 | Validation: 32 | Test: 17
Train: 69.57% | Validation: 19.88% | Test: 10.56%
Full: 29 | Train: 20 | Validation: 6 | Test: 3
Train: 68.97% | Validation: 20.69% | Test: 10.34%
Full: 198 | Train: 138 | Validation: 40 | Test: 20
Train: 69.70% | Validation: 20.20% | Test: 10.10%
Full: 206 | Train: 144 | Validation: 41 | Test: 21
Train: 69.90% | Validation: 19.90% | Test: 10.19%
Full: 53 | Train: 37 | Validation: 10 | Test: 6
Train: 69.81% | Validation: 18.87% | Test: 11.32%
Full: 250 | Train: 175 | Validation: 50 | Test: 25
Train: 70.00% | Validation: 20.00% | Test: 10.00%
Full: 67 | Train: 46 | Validation: 14 | Test: 7
Train: 68.66% | Validation: 20.90% | Test: 10.45%
Full: 347 | Train: 242 | Validation: 70 | Test: 35
Train: 69.74% | Validation: 20.17% | Test: 10.09%
Ful

Train: 67.74% | Validation: 19.35% | Test: 12.90%
Full: 63 | Train: 44 | Validation: 12 | Test: 7
Train: 69.84% | Validation: 19.05% | Test: 11.11%
Full: 459 | Train: 321 | Validation: 92 | Test: 46
Train: 69.93% | Validation: 20.04% | Test: 10.02%
Full: 134 | Train: 93 | Validation: 27 | Test: 14
Train: 69.40% | Validation: 20.15% | Test: 10.45%
Full: 751 | Train: 525 | Validation: 150 | Test: 76
Train: 69.91% | Validation: 19.97% | Test: 10.12%
Full: 26 | Train: 18 | Validation: 5 | Test: 3
Train: 69.23% | Validation: 19.23% | Test: 11.54%
Full: 421 | Train: 294 | Validation: 84 | Test: 43
Train: 69.83% | Validation: 19.95% | Test: 10.21%
Full: 59 | Train: 41 | Validation: 12 | Test: 6
Train: 69.49% | Validation: 20.34% | Test: 10.17%
Full: 29 | Train: 20 | Validation: 6 | Test: 3
Train: 68.97% | Validation: 20.69% | Test: 10.34%
Full: 535 | Train: 374 | Validation: 107 | Test: 54
Train: 69.91% | Validation: 20.00% | Test: 10.09%
Full: 430 | Train: 301 | Validation: 86 | Test: 43
Tra

Full: 612 | Train: 428 | Validation: 122 | Test: 62
Train: 69.93% | Validation: 19.93% | Test: 10.13%
Full: 46 | Train: 32 | Validation: 9 | Test: 5
Train: 69.57% | Validation: 19.57% | Test: 10.87%
Full: 88 | Train: 61 | Validation: 18 | Test: 9
Train: 69.32% | Validation: 20.45% | Test: 10.23%
Full: 205 | Train: 143 | Validation: 41 | Test: 21
Train: 69.76% | Validation: 20.00% | Test: 10.24%
Full: 249 | Train: 174 | Validation: 50 | Test: 25
Train: 69.88% | Validation: 20.08% | Test: 10.04%
Full: 478 | Train: 334 | Validation: 96 | Test: 48
Train: 69.87% | Validation: 20.08% | Test: 10.04%
Full: 40 | Train: 28 | Validation: 8 | Test: 4
Train: 70.00% | Validation: 20.00% | Test: 10.00%
Full: 48 | Train: 33 | Validation: 10 | Test: 5
Train: 68.75% | Validation: 20.83% | Test: 10.42%
Full: 288 | Train: 201 | Validation: 58 | Test: 29
Train: 69.79% | Validation: 20.14% | Test: 10.07%
Full: 30 | Train: 21 | Validation: 6 | Test: 3
Train: 70.00% | Validation: 20.00% | Test: 10.00%
Full: 2

(113857, 32640, 16652)

In [29]:
train.to_csv(os.path.join(save_path, "train.csv"), sep='\t', index=None)
val.to_csv(os.path.join(save_path, "val.csv"), sep='\t', index=None)
test.to_csv(os.path.join(save_path, "test.csv"), sep='\t', index=None)

In [30]:
# train["sequence"] = train.sequence.str.ljust(SEQUENCE_LENGTH, '0')
# val["sequence"] = val.sequence.str.ljust(SEQUENCE_LENGTH, '0')
# test["sequence"] = test.sequence.str.ljust(SEQUENCE_LENGTH, '0')

### Sequence to embeddings ids

In [5]:
def get_indicies(val, kmer2index, kmer_size):
    return [kmer2index.get(val[i:i+kmer_size], 0) for i in range(len(val)-(kmer_size-1))]

def convert_to_index(data, file_name):
    embedding_path = os.path.join(ROOT, "data_sources", file_name)
    embeddings = pd.read_csv(embedding_path, delimiter="\t")
    all_words = embeddings["words"]
    index2kmer = all_words.to_dict()
    kmer_size = len(index2kmer[1])
    kmer2index = inv_map = {v: k for k, v in index2kmer.items()}
    print("The size of vocabulary is {}".format(max(list(index2kmer.keys()))))
    data["sequence"] = data.sequence.swifter.apply(lambda x: get_indicies(x, kmer2index, kmer_size))
    return data, kmer_size

In [17]:
def store_data(embedding_path):
    print("Reading original data")
    train = pd.read_csv(os.path.join(save_path, "train.csv"), sep='\t')
    val = pd.read_csv(os.path.join(save_path, "val.csv"), sep='\t')
    test = pd.read_csv(os.path.join(save_path, "test.csv"), sep='\t')
    print("Lengths. Max: {} Min: {}".format(max([len(x) for x in train.sequence]), min([len(x) for x in train.sequence])))
    train_emb, _ = convert_to_index(train, embedding_path)
    val_emb, _ = convert_to_index(val, embedding_path)
    test_emb, kmer_size = convert_to_index(test, embedding_path)
    print("Embedded sequence lengths. Max: {} Min: {}".format(max([len(x) for x in train_emb.sequence]), 
                                                              min([len(x) for x in train_emb.sequence])))
    
    print("Indicies used. Max: {} Min: {}".format(max([max(x) for x in train_emb.sequence]), 
                                                  min([min(x) for x in train_emb.sequence])))

    tf_records_path = os.path.join(save_path, "{}_kmers".format(kmer_size))
    store_fn = save_as_npy #save_as_tfrecords_multithreaded
    store_fn(os.path.join(tf_records_path, "train"), train[["Label", "sequence", "Level_2"]])#, 
                                    #group_by_col="Level_2") 
    store_fn(os.path.join(tf_records_path, "val"), val[["Label", "sequence", "Level_2"]]) #,
                                   #group_by_col="Level_2") 
    store_fn(os.path.join(tf_records_path, "test"), test[["Label", "sequence", "Level_2"]]) #, 
                                    #group_by_col="Level_2") 

Saving down results

In [18]:
store_data("protVec_100d_3grams.csv")

Reading original data
Lengths. Max: 512 Min: 80
The size of vocabulary is 9047


Pandas Apply: 100%|██████████| 113857/113857 [00:13<00:00, 8308.54it/s]


The size of vocabulary is 9047


Pandas Apply: 100%|██████████| 32640/32640 [00:03<00:00, 8552.42it/s]


The size of vocabulary is 9047


Pandas Apply: 100%|██████████| 16652/16652 [00:02<00:00, 7967.83it/s]


Embedded sequence lengths. Max: 510 Min: 78
Indicies used. Max: 8001 Min: 1
Data was stored in ../../data/protein/classification/sample_512\3_kmers\train\data.npy
Data was stored in ../../data/protein/classification/sample_512\3_kmers\val\data.npy
Data was stored in ../../data/protein/classification/sample_512\3_kmers\test\data.npy


In [ ]:
store_data("2grams.csv")

Reading original data
Lengths. Max: 512 Min: 80
The size of vocabulary is 400


Pandas Apply: 100%|██████████| 113857/113857 [00:13<00:00, 8298.70it/s]


The size of vocabulary is 400


Pandas Apply: 100%|██████████| 32640/32640 [00:04<00:00, 8045.63it/s]


The size of vocabulary is 400


Pandas Apply: 100%|██████████| 16652/16652 [00:02<00:00, 7496.34it/s]


Embedded sequence lengths. Max: 511 Min: 79


In [ ]:
store_data("1grams.csv")

# Preprocessing done

In [ ]:
np.load("../../data/protein/classification/sample_512/1_kmers/train/data.npy")[0][0].shape